# DSA4264 Project

In [1]:
#Import modules
import emoji
import pandas as pd
import numpy as np
import string
from langdetect import detect

In [2]:
#Read in the data (STORE FILES IN SUBFOLDER CALLED DATA)
data2020_df = pd.read_csv('./data/Reddit-Threads_2020-2021.csv',  lineterminator='\n', encoding='utf8')
data2022_df = pd.read_csv('./data/Reddit-Threads_2022-2023.csv', lineterminator='\n', encoding='utf8')

#Remove comments which are [Removed] or [Deleted]
data2020_df = data2020_df[~data2020_df['text'].str.contains(r'\[removed\]', na=False)]
data2020_df = data2020_df[~data2020_df['text'].str.contains(r'\[deleted\]', na=False)]
data2022_df = data2022_df[~data2022_df['text'].str.contains(r'\[removed\]', na=False)]
data2022_df = data2022_df[~data2022_df['text'].str.contains(r'\[deleted\]', na=False)]


In [15]:
#print(len(data2020_df))
#print(len(data2022_df))

2663782
1840541


In [6]:
#Data cleaning

def clean_data(df):
  pd.options.mode.copy_on_write = True

  #Remove rows with empty review_text
  df = df[df['text'].notnull()]

  #Remove emoji rows
  df['text'] = df['text'].apply(lambda x: emoji.replace_emoji(x,''))

  #Remove punctuation
  df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

  #Remove all rows that has non ASCII characters
  df = df[df['text'].apply(lambda x: all(ord(c) < 128 for c in x))]

  #Set all to lower case
  df['text'] = df['text'].str.lower()

  '''
  #Remove rows that are not in English
  count = 1
  for index, row in df.iterrows():
      try:
        if detect(row['text']) != 'en':
          df.drop(index, inplace=True)
          print(row['text'])
          print(count)
          count += 1
      except:
        df.drop(index, inplace=True)
        print("error for", row['text'])
  '''

  df = df.reset_index(drop=True)

  return df

In [11]:
cleaned_2020 = clean_data(data2020_df)

In [12]:
cleaned_2022 = clean_data(data2022_df)

In [13]:
#Get random sample of 500000
length_2020 = len(cleaned_2020)
length_2022 = len(cleaned_2022)
size2020 = int(500000 * (length_2020/(length_2020 + length_2022)))
size2022 = 500000 - size2020

sample_2020 = cleaned_2020.sample(n=size2020, random_state=42)
sample_2022 = cleaned_2022.sample(n=size2022, random_state=42)

combined_df = pd.concat([sample_2020, sample_2022], axis=0)
combined_df.reset_index(drop=True, inplace=True)

print(len(combined_df)) 

500000


In [16]:
#print(size2020)
#print(size2022)

298642
201358


In [14]:
print(len(combined_df))
print(combined_df.describe())

500000
          text            timestamp   username  \
count   500000               500000     500000   
unique  480871               498422      42403   
top             2021-07-21 02:10:03  [deleted]   
freq      1423                    3      12521   

                                                     link    link_id  \
count                                              500000     500000   
unique                                             500000      70648   
top     /r/singapore/comments/11do8on/rsingapore_rando...  t3_homxdq   
freq                                                    1       1188   

        parent_id       id subreddit_id  \
count      500000   500000       500000   
unique     374430   500000            3   
top     t3_homxdq  jaayf9p     t5_2qh8c   
freq          559        1       450203   

                                               moderation  
count                                              500000  
unique                                       

In [15]:
#Save data to csv (Only need to run once)
#combined_df.to_csv('Reddit_cleaned.csv', index=False)